DENSE RETRIEVAL

In [ ]:
!pip install -U beir

     |████████████████████████████████| 49 kB 2.5 MB/s 
     |████████████████████████████████| 8.4 MB 5.2 MB/s 
     |████████████████████████████████| 4.4 MB 39.7 MB/s 
     |████████████████████████████████| 78 kB 8.0 MB/s 
     |████████████████████████████████| 378 kB 44.2 MB/s 
     |████████████████████████████████| 2.9 MB 35.7 MB/s 
     |████████████████████████████████| 3.3 MB 34.8 MB/s 
     |████████████████████████████████| 1.2 MB 50.8 MB/s 
     |████████████████████████████████| 56 kB 5.0 MB/s 
     |████████████████████████████████| 895 kB 48.9 MB/s 
     |████████████████████████████████| 636 kB 45.7 MB/s 
  Created wheel for pytrec-eval: filename=pytrec_eval-0.5-cp37-cp37m-linux_x86_64.whl size=266574 sha256=50bed722b48df4473d4e66b663bcc9c696c86d2b32a3bfa48164a9ba926624b0
  Stored in directory: /root/.cache/pip/wheels/42/96/77/0829b8b2606f90f61ba10a51277629d2b615604e122ee932f4
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any

In [ ]:
import pathlib, os
import time
import pandas as pd
import random
import requests
import json
import torch
import torch.multiprocessing as mp
from tqdm.notebook import tqdm
from tqdm.autonotebook import trange
from beir import util, LoggingHandler
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval import models
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES
from beir.datasets.data_loader import GenericDataLoader

hostname = 'localhost' 
dataset = 'scidocs'
index_name = dataset
url = 'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip'.format(dataset)
out_dir = os.path.join(os.getcwd(), 'datasets')
data_path = util.download_and_unzip(url, out_dir)
corpus, queries, qrels = GenericDataLoader(data_path).load(split='test') # 'test', 'train', 'dev'
corpus_ids = list(corpus.keys())
corpus_list = [corpus[doc_id] for doc_id in corpus_ids]

/content/datasets/scidocs.zip:   0%|          | 0.00/136M [00:00<?, ?iB/s]

  0%|          | 0/25657 [00:00<?, ?it/s]

In [ ]:
def eval_metrics(model_name, ndcg, _map, recall, precision):
  c_map = 'MAP@10'
  c_map_h = 'MAP@100'
  c_map_t = 'MAP@1000'
  c_ndcg = 'NDCG@10'
  c_ndcg_h = 'NDCG@100'
  c_ndcg_t = 'NDCG@1000'
  c_pre = 'P@10'
  c_pre_h = 'P@100'
  c_pre_t = 'P@1000'
  c_rec = 'Recall@10'
  c_rec_h = 'Recall@100'
  c_rec_t = 'Recall@1000'
  eval_dict = {
      c_map: [_map[c_map]], 
      c_map_h: [_map[c_map_h]],
      c_map_t: [_map[c_map_t]],
      c_ndcg: [ndcg[c_ndcg]], 
      c_ndcg_h: [ndcg[c_ndcg_h]],
      c_ndcg_t: [ndcg[c_ndcg_t]],
      c_pre: [precision[c_pre]], 
      c_pre_h: [precision[c_pre_h]],
      c_pre_t: [precision[c_pre_t]],
      c_rec: [recall[c_rec]],
      c_rec_h: [recall[c_rec_h]],
      c_rec_t: [recall[c_rec_t]]}
  eval_df = pd.DataFrame(data=eval_dict)
  eval_df.index = [model_name]
  return eval_df

In [ ]:
dr_all_mpnet_dot = EvaluateRetrieval(
    DRES(models.SentenceBERT('all-mpnet-base-v2'), batch_size=128), score_function='dot')
dr_all_drob_dot = EvaluateRetrieval(
    DRES(models.SentenceBERT('all-distilroberta-v1'), batch_size=128), score_function='dot')
dr_all_minilm_dot = EvaluateRetrieval(
    DRES(models.SentenceBERT('all-MiniLM-L12-v2'), batch_size=128), score_function='dot')

res_all_mpnet_dot = dr_all_mpnet_dot.retrieve(corpus, queries)
res_all_drob_dot = dr_all_drob_dot.retrieve(corpus, queries)
res_all_minilm_dot = dr_all_minilm_dot.retrieve(corpus, queries)

ndcg_d_amd, _map_d_amd, recall_d_amd, precision_d_amd = dr_all_mpnet_dot.evaluate(
    qrels, res_all_mpnet_dot, dr_all_mpnet_dot.k_values)
ndcg_d_add, _map_d_add, recall_d_add, precision_d_add = dr_all_drob_dot.evaluate(
    qrels, res_all_drob_dot, dr_all_drob_dot.k_values)
ndcg_d_amld, _map_d_amld, recall_d_amld, precision_d_amld = dr_all_minilm_dot.evaluate(
    qrels, res_all_minilm_dot, dr_all_minilm_dot.k_values)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Batches:   0%|          | 0/201 [00:00<?, ?it/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/201 [00:00<?, ?it/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/201 [00:00<?, ?it/s]

In [ ]:
dr_mqa_mpnet_dot = EvaluateRetrieval(
    DRES(models.SentenceBERT('multi-qa-mpnet-base-dot-v1'), batch_size=128), score_function='dot')
dr_mqa_minilm_cos = EvaluateRetrieval(
    DRES(models.SentenceBERT('multi-qa-MiniLM-L6-cos-v1'), batch_size=128), score_function='cos_sim')
dr_mqa_dist_dot = EvaluateRetrieval(
    DRES(models.SentenceBERT('multi-qa-distilbert-dot-v1'), batch_size=128), score_function='dot')
dr_mqa_dist_cos = EvaluateRetrieval(
    DRES(models.SentenceBERT('multi-qa-distilbert-cos-v1'), batch_size=128), score_function='cos_sim')

res_mqa_mpnet_dot = dr_mqa_mpnet_dot.retrieve(corpus, queries)
res_mqa_minilm_cos = dr_mqa_minilm_cos.retrieve(corpus, queries)
res_mqa_dist_dot = dr_mqa_dist_dot.retrieve(corpus, queries)
res_mqa_dist_cos = dr_mqa_dist_cos.retrieve(corpus, queries)

ndcg_d_mmd, _map_d_mmd, recall_d_mmd, precision_d_mmd = dr_mqa_mpnet_dot.evaluate(
    qrels, res_mqa_mpnet_dot, dr_mqa_mpnet_dot.k_values)
ndcg_d_mlc, _map_d_mlc, recall_d_mlc, precision_d_mlc = dr_mqa_minilm_cos.evaluate(
    qrels, res_mqa_minilm_cos, dr_mqa_minilm_cos.k_values)
ndcg_d_mdd, _map_d_mdd, recall_d_mdd, precision_d_mdd = dr_mqa_dist_dot.evaluate(
    qrels, res_mqa_dist_dot, dr_mqa_dist_dot.k_values)
ndcg_d_mdc, _map_d_mdc, recall_d_mdc, precision_d_mdc = dr_mqa_dist_cos.evaluate(
    qrels, res_mqa_dist_cos, dr_mqa_dist_cos.k_values)

DEBUG:filelock:Attempting to acquire lock 139766561680016 on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-dot-v1/.gitattributes.lock
DEBUG:filelock:Lock 139766561680016 acquired on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-dot-v1/.gitattributes.lock


Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 139766561680016 on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-dot-v1/.gitattributes.lock
DEBUG:filelock:Lock 139766561680016 released on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-dot-v1/.gitattributes.lock
DEBUG:filelock:Attempting to acquire lock 139766561667728 on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-dot-v1/README.md.lock
DEBUG:filelock:Lock 139766561667728 acquired on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-dot-v1/README.md.lock


Downloading:   0%|          | 0.00/8.40k [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 139766561667728 on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-dot-v1/README.md.lock
DEBUG:filelock:Lock 139766561667728 released on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-dot-v1/README.md.lock
DEBUG:filelock:Attempting to acquire lock 139766744243216 on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-dot-v1/config.json.lock
DEBUG:filelock:Lock 139766744243216 acquired on /root/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-mpnet-base-dot-v1/config.json.lock


...

Batches:   0%|          | 0/201 [00:00<?, ?it/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/201 [00:00<?, ?it/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/201 [00:00<?, ?it/s]

In [ ]:
dr_msm_dist_cls = EvaluateRetrieval(
    DRES(models.SentenceBERT('msmarco-distilbert-base-tas-b'), batch_size=128), score_function='dot')
dr_msm_dist_dot = EvaluateRetrieval(
    DRES(models.SentenceBERT('msmarco-distilbert-dot-v5'), batch_size=128), score_function='dot')
dr_msm_con_dot = EvaluateRetrieval(
    DRES(models.SentenceBERT('msmarco-bert-co-condensor'), batch_size=128), score_function='dot')
dr_msm_ance_dot = EvaluateRetrieval(
    DRES(models.SentenceBERT('msmarco-roberta-base-ance-firstp')), score_function='dot')

res_msm_dist_cls = dr_msm_dist_cls.retrieve(corpus, queries)
res_msm_dist_dot = dr_msm_dist_dot.retrieve(corpus, queries)
res_msm_con_dot = dr_msm_con_dot.retrieve(corpus, queries)
res_msm_ance_dot = dr_msm_ance_dot.retrieve(corpus, queries)

ndcg_d_sdc, _map_d_sdc, recall_d_sdc, precision_d_sdc = dr_msm_dist_cls.evaluate(
    qrels, res_msm_dist_cls, dr_msm_dist_cls.k_values)
ndcg_d_sdd, _map_d_sdd, recall_d_sdd, precision_d_sdd = dr_msm_dist_dot.evaluate(
    qrels, res_msm_dist_dot, dr_msm_dist_dot.k_values)
ndcg_d_scd, _map_d_scd, recall_d_scd, precision_d_scd = dr_msm_con_dot.evaluate(
    qrels, res_msm_con_dot, dr_msm_con_dot.k_values)
ndcg_d_sad, _map_d_sad, recall_d_sad, precision_d_sad = dr_msm_ance_dot.evaluate(
    qrels, res_msm_ance_dot, dr_msm_ance_dot.k_values)

DEBUG:filelock:Attempting to acquire lock 139766154719504 on /root/.cache/torch/sentence_transformers/sentence-transformers_msmarco-distilbert-base-tas-b/.gitattributes.lock
DEBUG:filelock:Lock 139766154719504 acquired on /root/.cache/torch/sentence_transformers/sentence-transformers_msmarco-distilbert-base-tas-b/.gitattributes.lock


Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 139766154719504 on /root/.cache/torch/sentence_transformers/sentence-transformers_msmarco-distilbert-base-tas-b/.gitattributes.lock
DEBUG:filelock:Lock 139766154719504 released on /root/.cache/torch/sentence_transformers/sentence-transformers_msmarco-distilbert-base-tas-b/.gitattributes.lock
DEBUG:filelock:Attempting to acquire lock 139766901327760 on /root/.cache/torch/sentence_transformers/sentence-transformers_msmarco-distilbert-base-tas-b/README.md.lock
DEBUG:filelock:Lock 139766901327760 acquired on /root/.cache/torch/sentence_transformers/sentence-transformers_msmarco-distilbert-base-tas-b/README.md.lock


Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 139766901327760 on /root/.cache/torch/sentence_transformers/sentence-transformers_msmarco-distilbert-base-tas-b/README.md.lock
DEBUG:filelock:Lock 139766901327760 released on /root/.cache/torch/sentence_transformers/sentence-transformers_msmarco-distilbert-base-tas-b/README.md.lock
DEBUG:filelock:Attempting to acquire lock 139766561681360 on /root/.cache/torch/sentence_transformers/sentence-transformers_msmarco-distilbert-base-tas-b/config.json.lock
DEBUG:filelock:Lock 139766561681360 acquired on /root/.cache/torch/sentence_transformers/sentence-transformers_msmarco-distilbert-base-tas-b/config.json.lock


...

Batches:   0%|          | 0/201 [00:00<?, ?it/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/201 [00:00<?, ?it/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/201 [00:00<?, ?it/s]

In [ ]:
eval_metrics('all-mpnet-base-v2', ndcg_d_amd, _map_d_amd, recall_d_amd, precision_d_amd)

,MAP@10,MAP@100,MAP@1000,NDCG@10,NDCG@100,NDCG@1000,P@10,P@100,P@1000,Recall@10,Recall@100,Recall@1000
all-mpnet-base-v2,0.14426,0.1718,0.1758,0.23765,0.33664,0.39481,0.1269,0.0271,0.00409,0.25712,0.54997,0.82972


In [ ]:
eval_metrics('all-distilroberta-v1', ndcg_d_add, _map_d_add, recall_d_add, precision_d_add)

,MAP@10,MAP@100,MAP@1000,NDCG@10,NDCG@100,NDCG@1000,P@10,P@100,P@1000,Recall@10,Recall@100,Recall@1000
all-distilroberta-v1,0.1303,0.15713,0.16093,0.21694,0.31561,0.37487,0.1144,0.02564,0.00398,0.23192,0.52075,0.80807


In [ ]:
eval_metrics('all-MiniLM-L12-v2', ndcg_d_amld, _map_d_amld, recall_d_amld, precision_d_amld)

,MAP@10,MAP@100,MAP@1000,NDCG@10,NDCG@100,NDCG@1000,P@10,P@100,P@1000,Recall@10,Recall@100,Recall@1000
all-MiniLM-L12-v2,0.13168,0.15766,0.16136,0.21818,0.31303,0.37175,0.1132,0.02495,0.00389,0.22932,0.50598,0.79053


In [ ]:
eval_metrics('multi-qa-mpnet-base-dot-v1', ndcg_d_mmd, _map_d_mmd, recall_d_mmd, precision_d_mmd)

,MAP@10,MAP@100,MAP@1000,NDCG@10,NDCG@100,NDCG@1000,P@10,P@100,P@1000,Recall@10,Recall@100,Recall@1000
multi-qa-mpnet-base-dot-v1,0.09721,0.11263,0.11523,0.16627,0.22982,0.28038,0.0839,0.0175,0.00297,0.17047,0.35492,0.60205


In [ ]:
eval_metrics('multi-qa-MiniLM-L6-cos-v1', ndcg_d_mlc, _map_d_mlc, recall_d_mlc, precision_d_mlc)

,MAP@10,MAP@100,MAP@1000,NDCG@10,NDCG@100,NDCG@1000,P@10,P@100,P@1000,Recall@10,Recall@100,Recall@1000
multi-qa-MiniLM-L6-cos-v1,0.0921,0.10629,0.10859,0.1578,0.21807,0.26593,0.0798,0.01667,0.00283,0.1617,0.33848,0.57345


In [ ]:
eval_metrics('multi-qa-distilbert-dot-v1', ndcg_d_mdd, _map_d_mdd, recall_d_mdd, precision_d_mdd)

,MAP@10,MAP@100,MAP@1000,NDCG@10,NDCG@100,NDCG@1000,P@10,P@100,P@1000,Recall@10,Recall@100,Recall@1000
multi-qa-distilbert-dot-v1,0.09098,0.10745,0.10991,0.15781,0.22602,0.27463,0.0804,0.01784,0.00296,0.16287,0.36245,0.60043


In [ ]:
eval_metrics('multi-qa-distilbert-cos-v1', ndcg_d_mdc, _map_d_mdc, recall_d_mdc, precision_d_mdc)

,MAP@10,MAP@100,MAP@1000,NDCG@10,NDCG@100,NDCG@1000,P@10,P@100,P@1000,Recall@10,Recall@100,Recall@1000
multi-qa-distilbert-cos-v1,0.09239,0.1091,0.11162,0.16004,0.2277,0.27644,0.0822,0.01796,0.00296,0.1669,0.36423,0.60232


In [ ]:
eval_metrics('msmarco-distilbert-base-tas-b', ndcg_d_sdc, _map_d_sdc, recall_d_sdc, precision_d_sdc)

,MAP@10,MAP@100,MAP@1000,NDCG@10,NDCG@100,NDCG@1000,P@10,P@100,P@1000,Recall@10,Recall@100,Recall@1000
msmarco-distilbert-base-tas-b,0.08543,0.10013,0.10248,0.14859,0.21063,0.25835,0.0748,0.01646,0.0028,0.15185,0.33462,0.56763


In [ ]:
eval_metrics('msmarco-distilbert-dot-v5', ndcg_d_sdd, _map_d_sdd, recall_d_sdd, precision_d_sdd)

,MAP@10,MAP@100,MAP@1000,NDCG@10,NDCG@100,NDCG@1000,P@10,P@100,P@1000,Recall@10,Recall@100,Recall@1000
msmarco-distilbert-dot-v5,0.08077,0.09356,0.09566,0.14048,0.19639,0.24117,0.071,0.01522,0.0026,0.14425,0.30865,0.5281


In [ ]:
eval_metrics('msmarco-bert-co-condensor', ndcg_d_scd, _map_d_scd, recall_d_scd, precision_d_scd)

,MAP@10,MAP@100,MAP@1000,NDCG@10,NDCG@100,NDCG@1000,P@10,P@100,P@1000,Recall@10,Recall@100,Recall@1000
msmarco-bert-co-condensor,0.08013,0.09277,0.09489,0.13946,0.19551,0.24079,0.0718,0.01535,0.00263,0.14543,0.31172,0.53407


In [ ]:
eval_metrics('msmarco-roberta-base-ance-firstp', ndcg_d_sad, _map_d_sad, recall_d_sad, precision_d_sad)

,MAP@10,MAP@100,MAP@1000,NDCG@10,NDCG@100,NDCG@1000,P@10,P@100,P@1000,Recall@10,Recall@100,Recall@1000
msmarco-roberta-base-ance-firstp,0.06951,0.08045,0.08233,0.12187,0.17146,0.21404,0.0609,0.01324,0.00236,0.1236,0.26928,0.479


end of fun.